In [60]:
rm(list = ls())

In [61]:
library("data.table")
library("stargazer")

In [62]:
sample.coverage = fread("notebook_data/vep_SNV.merged.table")
sample.coverage[,ID:=paste0(CHROM,"_",POS,"_",REF,"_",ALT)]
options(repr.matrix.max.cols = 80)
head(sample.coverage)

CHROM,POS,REF,ALT,TYPE,CALLER,TUMOR_DP,TUMOR_AD_REF,TUMOR_AD_ALT,FILTERED,MSK_MVL,Allele,Consequence,IMPACT,SYMBOL,Gene,Feature_type,Feature,BIOTYPE,EXON,INTRON,HGVSc,HGVSp,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,Existing_variation,DISTANCE,STRAND,FLAGS,VARIANT_CLASS,SYMBOL_SOURCE,HGNC_ID,CANONICAL,TSL,APPRIS,CCDS,ENSP,SWISSPROT,TREMBL,UNIPARC,GENE_PHENO,SIFT,PolyPhen,DOMAINS,HGVS_OFFSET,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF,AA_AF,EA_AF,gnomAD_AF,gnomAD_AFR_AF,gnomAD_AMR_AF,gnomAD_ASJ_AF,gnomAD_EAS_AF,gnomAD_FIN_AF,gnomAD_NFE_AF,gnomAD_OTH_AF,gnomAD_SAS_AF,MAX_AF,MAX_AF_POPS,CLIN_SIG,SOMATIC,PHENO,PUBMED,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,ID
1,898012,G,T,SNP,VARDICT,99,96,3,NO,.,T,downstream_gene_variant,MODIFIER,KLHL17,ENSG00000187961,Transcript,ENST00000463212,retained_intron,.,.,.,.,.,.,.,.,.,.,154,1,.,SNV,HGNC,24023,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,1_898012_G_T
1,898012,G,T,SNP,VARDICT,99,96,3,NO,.,T,intron_variant,MODIFIER,KLHL17,ENSG00000187961,Transcript,ENST00000338591,protein_coding,.,5/11,ENST00000338591.3:c.829-72G>T,.,.,.,.,.,.,.,.,1,.,SNV,HGNC,24023,YES,.,.,CCDS30550.1,ENSP00000343930,Q6TDP4,Q0VGE6&B3KXL7,UPI00001DFBF0,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,1_898012_G_T
1,898012,G,T,SNP,VARDICT,99,96,3,NO,.,T,regulatory_region_variant,MODIFIER,.,.,RegulatoryFeature,ENSR00000344307,CTCF_binding_site,.,.,.,.,.,.,.,.,.,.,.,.,.,SNV,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,1_898012_G_T
1,898012,G,T,SNP,VARDICT,99,96,3,NO,.,T,upstream_gene_variant,MODIFIER,KLHL17,ENSG00000187961,Transcript,ENST00000466300,nonsense_mediated_decay,.,.,.,.,.,.,.,.,.,.,95,1,cds_start_NF,SNV,HGNC,24023,.,.,.,.,ENSP00000463694,.,.,UPI000268AE1E,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,1_898012_G_T
1,898012,G,T,SNP,VARDICT,99,96,3,NO,.,T,upstream_gene_variant,MODIFIER,KLHL17,ENSG00000187961,Transcript,ENST00000481067,retained_intron,.,.,.,.,.,.,.,.,.,.,920,1,.,SNV,HGNC,24023,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,1_898012_G_T
1,898012,G,T,SNP,VARDICT,99,96,3,NO,.,T,upstream_gene_variant,MODIFIER,NOC2L,ENSG00000188976,Transcript,ENST00000327044,protein_coding,.,.,.,.,.,.,.,.,.,.,3342,-1,.,SNV,HGNC,24517,YES,.,.,CCDS3.1,ENSP00000317992,Q9Y3T9,.,UPI000041820C,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,1_898012_G_T


In [53]:
genomeLength = 38874504/1e6

var_caller = as.list(unique(sample.coverage[,"CALLER"]))$CALLER
annotation = c("stop_gained", "stop_lost", "start_lost",
               "missense_variant", "nonsynonymous_variant",
               "splice_acceptor_variant", "splice_donor_variant",
               "splice_donor_5th_base_variant", "splice_site_variant",
               "splicing_variant", "frameshift_variant")

dt1 = unique(sample.coverage[CALLER %in% var_caller
                     & Consequence %in% annotation, .(CHROM, POS),
                     by=.(ID, CALLER)])[,.(.N,"TMB"=.N/genomeLength),by=.(CALLER)]

dt2 = unique(sample.coverage[CALLER %in% var_caller
                     & Consequence %in% annotation, .(CALLER),
                     by=.(ID)][,.(ID)])[,.(.N)]
dt2[,"CALLER":="ALL"]
setcolorder(dt2, neworder = c("CALLER", "N"))
dt = dt2[,.(CALLER, N, "TMB"=N/genomeLength)]

str_annot = paste(gsub("_", "-", annotation), collapse = ", ")
dt.TMB = rbind(dt1, dt)
stargazer(unique(dt.TMB), summary = FALSE, type = "text", rownames = F,
          title = "Tumor mutation burden (TMB)",
          digit.separator = "",
          notes = c(paste0("1. Variant callers included: ",
                           tolower(paste(as.list(unique(sample.coverage[,"CALLER"]))$CALLER,
                                         collapse = ", "))),
                    paste0("2. Variant types: ",
                           tolower(paste(as.list(unique(sample.coverage[,"VARIANT_CLASS"]))$VARIANT_CLASS,
                                         collapse=", "))),
                    paste0("3. Only all coding variants (all subchilds of nonsynonymous variants annotation)")))



Tumor mutation burden (TMB)
CALLER                                    N                         TMB           
----------------------------------------------------------------------------------
VARDICT                                  6603                     169.854         
STRELKA                                  680                      17.492          
MUTECT2                                  1324                     34.058          
ALL                                      8188                     210.626         
----------------------------------------------------------------------------------
1. Variant callers included: vardict, mutect2, strelka                            
2. Variant types: snv, indel, insertion, deletion, substitution                   
3. Only all coding variants (all subchilds of nonsynonymous variants annotation)  


In [63]:
dt = unique(sample.coverage[,.(ID,CALLER,VARIANT_CLASS)])
dt.typevars = dt[,.("CALLERCOUNT"=length(unique(c(CALLER))),.N),
                 by=.("CALLERS"=CALLER,VARIANT_CLASS)][order(CALLERS,-VARIANT_CLASS)]

dt = unique(sample.coverage[,.(ID,CALLER,VARIANT_CLASS)])
dt = dt[,.("CALLERS"="ALL","CALLERCOUNT"=length(unique(c(CALLER))),.N),
        by=(VARIANT_CLASS)]
setcolorder(dt, neworder = c("CALLERS", "VARIANT_CLASS", "CALLERCOUNT","N"))
dt.typecensus = rbind(dt, dt.typevars)

dt = unique(sample.coverage[,.(ID,CALLER)])
dt.allcallers = dt[,.("VARIANT_CLASS"="All_types","CALLERCOUNT"=1,.N), by=.("CALLERS"=CALLER) ][order(CALLERS)]

dt.callercensus = rbind(dt.allcallers,dt.typecensus)[order(CALLERS,VARIANT_CLASS,CALLERCOUNT)]
stargazer(unique(dt.callercensus), summary = FALSE, type = "text", rownames = F,
          title = "Variant class summary",
          digit.separator = "",
          notes = c(paste0("1. A summary of variant classes devided by variant class and variant caller"),
                    paste0("2. Variant callers included: ",
                           tolower(paste(as.list(unique(sample.coverage[,"CALLER"]))$CALLER,
                                         collapse = ", "))),
                    paste0("3. Variant types: ",
                           tolower(paste(as.list(unique(sample.coverage[,"VARIANT_CLASS"]))$VARIANT_CLASS,
                                         collapse=", ")))))


Variant class summary
CALLERS                 VARIANT_CLASS        CALLERCOUNT             N        
------------------------------------------------------------------------------
ALL                          SNV                  3                5762       
ALL                       deletion                3                 731       
ALL                         indel                 1                1733       
ALL                       insertion               3                7085       
ALL                     substitution              1                 472       
MUTECT2                   All_types               1                2353       
MUTECT2                      SNV                  1                 482       
MUTECT2                   deletion                1                 283       
MUTECT2                   insertion               1                1588       
STRELKA                   All_types               1                2267       
STRELKA                      

In [67]:
var_caller = as.list(unique(sample.coverage[,"CALLER"]))$CALLER
var_caller_combn = do.call("c", lapply(seq_along(var_caller),
                                       function(i) {combn(var_caller, i, simplify = F)}))

dt = unique(sample.coverage[,.(ID,CALLER,VARIANT_CLASS)])
dt = dt[,.("CALLERS"=paste(unique(c(CALLER)),collapse = "-"),
                        "CALLERCOUNT"=length(unique(c(CALLER)))),
                     by=.(ID)][,.(.N),
                               by=.(CALLERS,CALLERCOUNT)
                              ]

dt.venn = dt[order(CALLERCOUNT,CALLERS)]
stargazer(unique(dt.venn), summary = FALSE, type = "text",
          title = "Variant caller summary",
          digit.separator = "", rownames = F,
          notes = c(paste0("1. A summary of exclusive variant types ",
                           "devided by variant callers that identified them"),
                    paste0("2. Variant callers included: ",
                           tolower(paste(as.list(unique(sample.coverage[,"CALLER"]))$CALLER,
                                         collapse = ", "))),
                    paste0("3. Variant types: ",
                           tolower(paste(as.list(unique(sample.coverage[,"VARIANT_CLASS"]))$VARIANT_CLASS,
                                         collapse=", ")))))


Variant caller summary
CALLERS                                         CALLERCOUNT                   N          
-----------------------------------------------------------------------------------------
MUTECT2                                              1                      2035         
STRELKA                                              1                      1823         
VARDICT                                              1                      10505        
STRELKA-MUTECT2                                      2                       19          
VARDICT-MUTECT2                                      2                       233         
VARDICT-STRELKA                                      2                       359         
VARDICT-STRELKA-MUTECT2                              3                       66          
-----------------------------------------------------------------------------------------
1. A summary of exclusive variant types devided by variant callers that iden

In [40]:
var_caller_combn = do.call("c", lapply(seq_along(var_caller),
                                       function(i) {combn(var_caller, i, simplify = F)}))

dt = unique(sample.coverage[,.(ID,CALLER,VARIANT_CLASS)])
dt = dt[,.("CALLERS"=paste(unique(c(CALLER)),collapse = "-"),
                        "CALLERCOUNT"=length(unique(c(CALLER))),
                        VARIANT_CLASS),
                     by=.(ID)][,.(.N),
                               by=.(CALLERS,VARIANT_CLASS,CALLERCOUNT)
                              ]

dt.venn = dt[order(CALLERCOUNT,CALLERS,VARIANT_CLASS)]
stargazer(unique(dt.venn), summary = FALSE, type = "text", rownames = F,
          title = "Variant caller summary by class",
          digit.separator = "",
          notes = c(paste0("1. A summary of exclusive variant types ",
                           "devided by variant callers that identified them"),
                    paste0("2. Variant callers included: ",
                           tolower(paste(as.list(unique(sample.coverage[,"CALLER"]))$CALLER,
                                         collapse = ", "))),
                    paste0("3. Variant types: ",
                           tolower(paste(as.list(unique(sample.coverage[,"VARIANT_CLASS"]))$VARIANT_CLASS,
                                         collapse=", ")))))


Variant caller summary by class
CALLERS                              VARIANT_CLASS          CALLERCOUNT            N      
------------------------------------------------------------------------------------------
MUTECT2                                   SNV                    1                376     
MUTECT2                                 deletion                 1                241     
MUTECT2                                insertion                 1               1418     
STRELKA                                   SNV                    1               1700     
STRELKA                                 deletion                 1                91      
STRELKA                                insertion                 1                32      
VARDICT                                   SNV                    1               2712     
VARDICT                                 deletion                 1                303     
VARDICT                                  indel           